# Fine Tune GPT-2






In [ ]:
!pip install pretty_midi tensorflow

In [ ]:
pip install pydub

In [ ]:
!apt-get update
!apt-get install fluidsynth

In [ ]:
pip install pyttsx3

In [ ]:
!apt-get update && apt-get install -y espeak


In [ ]:
!apt-get install -y musescore3 # Install MuseScore3


In [ ]:
pip install gtts


In [ ]:
import pyttsx3
from gtts import gTTS
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import random
from music21 import converter, instrument, note, stream
from pydub import AudioSegment
import subprocess
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from google.colab import drive
from music21 import converter, instrument, note, stream
from pydub import AudioSegment
import pretty_midi
from gtts import gTTS
import subprocess

# Suppress unnecessary TensorFlow warnings to make logs cleaner
tf.get_logger().setLevel('ERROR')

# Mount Google Drive to access stored models and data
drive.mount('/content/drive')

# Load the fine-tuned GPT-2 model and tokenizer from Google Drive to generate poetry
model_path = '/content/drive/MyDrive/gpt2-finetuned-poetry'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Load the LSTM model for music generation, along with the pitch mappings, from Google Drive
lstm_model_path = '/content/drive/MyDrive/final_lstm_music_model.keras'
pitch_to_idx_path = '/content/drive/MyDrive/pitch_to_index_lstm.pkl'
idx_to_pitch_path = '/content/drive/MyDrive/index_to_pitch_lstm.pkl'
seed_path = '/content/drive/MyDrive/lstm_train_seeds.pkl'

# Helper function to load the LSTM model and pitch mappings
def load_lstm_model_and_mappings():
    lstm_model = load_model(lstm_model_path, compile=False)
    with open(pitch_to_idx_path, 'rb') as f:
        pitch_to_idx = pickle.load(f)
    with open(idx_to_pitch_path, 'rb') as f:
        idx_to_pitch = pickle.load(f)
    return lstm_model, pitch_to_idx, idx_to_pitch

# Helper function to load pre-generated seed sequences for LSTM model input
def load_seed_sequences(seeds_file=seed_path):
    with open(seeds_file, 'rb') as f:
        seed_sequences = pickle.load(f)
    return seed_sequences

# Load LSTM model and data mappings
lstm_model, pitch_to_idx, idx_to_pitch = load_lstm_model_and_mappings()
seed_sequences = load_seed_sequences()

# Function to generate poetry using GPT-2 Fine tuned model
def generate_poem():
    print("Generating poem...")
    # Ask the user for poet and genre selection, plus a seed text to start generating
    selected_poet = input("Enter the poet token (e.g., <Poet:Shakespeare>): ").strip()
    selected_genre = input("Enter the genre token (e.g., <Genre:Romantic>): ").strip()
    seed_text = input("Enter the starting seed text: ").strip()
    input_text = f'{selected_poet} {selected_genre} {seed_text}'

    # Tokenize the input and generate poetry based on it
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(
        inputs['input_ids'],
        max_length=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text and clean up unnecessary tokens
    generated_poem = tokenizer.decode(outputs[0], skip_special_tokens=False)
    cleaned_poem = generated_poem.replace(selected_poet, '').replace(selected_genre, '').replace('[PAD]', '').replace('<LINE_BREAK>', '\n').strip()

    return cleaned_poem

# Function to generate a music sequence using the LSTM model
def generate_music_sequence(model, seeds, idx_to_pitch, seq_length=64, generation_steps=200, temperature=1.0, activation_threshold=0.5):
    # Pick a random seed sequence and initialize with it
    seed_sequence = random.choice(seeds)
    generated_sequence = seed_sequence.copy()

    # Generate new notes by predicting based on the current sequence
    for step in range(generation_steps):
        input_array = np.zeros((1, seq_length, len(idx_to_pitch)), dtype=np.float32)
        for t in range(seq_length):
            if len(generated_sequence) >= seq_length:
                current_step = generated_sequence[-seq_length + t]
            else:
                current_step = generated_sequence[:t+1]
            for pitch_idx in current_step:
                if 0 <= pitch_idx < len(idx_to_pitch):
                    input_array[0, t, pitch_idx] = 1.0  # Multi-hot encode each note

        predictions = model.predict(input_array, verbose=0)[0, -1]
        scaled_predictions = np.log(predictions + 1e-8) / temperature
        exp_predictions = np.exp(scaled_predictions)
        probabilities = exp_predictions / np.sum(exp_predictions)

        # Determine active pitches by thresholding probabilities
        active_pitches = [idx for idx, prob in enumerate(probabilities) if prob > activation_threshold]
        if not active_pitches:
            active_pitches = [np.argmax(probabilities)]

        generated_sequence.append(active_pitches)

    return generated_sequence

# Function to convert the generated music sequence into a MIDI file
def convert_sequence_to_midi(generated_sequence, idx_to_pitch, output_file='lstm_generated_music.mid', step_duration=0.25, velocity=100):
    midi_obj = pretty_midi.PrettyMIDI()
    piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)

    # Keep track of time for each note
    current_time = 0.0
    active_notes = {}

    for pitches in generated_sequence:
        pitches_to_remove = set(active_notes.keys()) - set(pitches)
        for pitch in pitches_to_remove:
            note = active_notes[pitch]
            note.end = current_time
            piano.notes.append(note)
            del active_notes[pitch]

        for pitch in pitches:
            if pitch not in active_notes:
                new_note = pretty_midi.Note(
                    velocity=velocity,
                    pitch=pitch,
                    start=current_time,
                    end=current_time + step_duration
                )
                active_notes[pitch] = new_note
            else:
                active_notes[pitch].end = current_time + step_duration

        current_time += step_duration

    for pitch, note in active_notes.items():
        note.end = current_time
        piano.notes.append(note)

    midi_obj.instruments.append(piano)
    midi_obj.write(output_file)

# Function to convert MIDI to WAV format using FluidSynth
def convert_midi_to_wav(midi_path, wav_path):
    cmd = ['fluidsynth', '-ni', '/path/to/FluidR3_GM.sf2', midi_path, '-F', wav_path, '-r', '44100']
    subprocess.run(cmd, check=True)

# Generate music, save it as MIDI, then convert to MP3
def generate_and_save_music(complete_poem, output_midi='background_music.mid'):
    generated_sequence = generate_music_sequence(lstm_model, seed_sequences, idx_to_pitch)
    convert_sequence_to_midi(generated_sequence, idx_to_pitch, output_file=output_midi)

    midi_path = output_midi
    wav_path = 'output.wav'
    convert_midi_to_wav(midi_path, wav_path)
    sound = AudioSegment.from_wav(wav_path)
    sound.export('background_music.mp3', format='mp3')

# Function to read the generated poem using gTTS, turning it into audio
def read_poetry_poetically(poem, output_file='poem_audio.mp3'):
    print("Reading poem in a poetic way...")

    lines = poem.split('. ')
    audio_segments = []

    for idx, line in enumerate(lines):
        poetic_line = f'{line.strip()}.'

        tts = gTTS(text=poetic_line, lang='en')
        temp_file = f'temp_line_{idx}.mp3'
        tts.save(temp_file)

        audio_segment = AudioSegment.from_mp3(temp_file)
        audio_segments.append(audio_segment)

        # Add a pause after each line for rhythm variation
        pause_duration = 1000 if idx % 2 == 0 else 1500
        silence = AudioSegment.silent(duration=pause_duration)
        audio_segments.append(silence)

        os.remove(temp_file)

    combined_audio = audio_segments[0]
    for segment in audio_segments[1:]:
        combined_audio += segment

    combined_audio.export(output_file, format='mp3')
    print(f"Poetry audio saved as {output_file}.")

# Function to loop background music to match the poem's duration
def loop_background_music(music, target_duration):
    loops_needed = int(target_duration / len(music)) + 1
    extended_music = music * loops_needed
    return extended_music[:target_duration]

# Main execution flow for poetry and music generation
if __name__ == "__main__":
    complete_poem = generate_poem()
    print("Complete Poem:")
    print(complete_poem)

    generate_and_save_music(complete_poem)
    read_poetry_poetically(complete_poem, 'poem_audio.mp3')

    poem_audio = AudioSegment.from_mp3('poem_audio.mp3')
    music_background = AudioSegment.from_mp3('background_music.mp3')

    extended_music_background = loop_background_music(music_background, len(poem_audio))

    combined_audio = poem_audio.overlay(extended_music_background)
    combined_audio.export('final_output.mp3', format='mp3')
    print("Final output saved as 'final_output.mp3'.")
